# **SpaceX  Falcon 9 first stage Landing Prediction**


Hey, this is Chakshu and this is one of my first project. In this set of the Project, we will collect the Data from various sources to make it ready for Data Analysis Project. Our Key Objectives are:

Objectives:
1. Request to the SpaceX API
2. Clean the requested data




![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/lab_v2/images/landing_1.gif)


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/lab_v2/images/crash.gif)


## Importing Libraries and Defining Auxiliary Variables

In [27]:
import requests                         # to make HTTP request; gets Data from URL

import numpy as np, pandas as pd        # managing the Data

from datetime import datetime                         # Lets us represent Dates

### Sources for Data:

Rocket Data: https://docs.spacexdata.com/#16c58b5e-44de-4183-b858-0fae51d242a5

SpaceX V4 Data: https://github.com/r-spacex/SpaceX-API/blob/master/docs/rockets/v4/query.md


In [70]:
# to extract Launch data between certain Start and End Date:



def get_launch_data_between_dates(start_input, end_input):

    launch_url = "https://api.spacexdata.com/v4/launches"
    
    api_response = requests.get(launch_url)

    if api_response.status_code != 200:
        print("Occured Error in API response...")
        return

    else:
        #raw_launch_data = response.json()
        launch_data = pd.json_normalize(api_response.json())
        launch_data['date_utc'] = launch_data['date_utc'].str[:10]
        launch_data['date_utc'] = pd.to_datetime(launch_data['date_utc'])

        start_date = pd.to_datetime(start_input)
        end_date = pd.to_datetime(end_input)
        print(f" start____ {start_date}  |   end_____{end_date}")

        launch_data_cleared = launch_data[(launch_data['date_utc'] >= start_date) & (launch_data['date_utc'] <= end_date)]
        return launch_data_cleared

        


launch_data = get_launch_data_between_dates(start_input="2000-01-01" , end_input="2023-01-01")
print(launch_data.shape)

 start____ 2000-01-01 00:00:00  |   end_____2023-01-01 00:00:00
(205, 43)


Since there are references to the Values in Some rows, so we need to fetch the value:



In [71]:
retained_feature_raw_data = launch_data[[
                                'net',
                                'window' , 
                                'rocket' , 
                                'success' , 
                                'failures' , 
                                'details' , 
                                'crew' ,
                                'ships' , 
                                'capsules' ,
                                'payloads' , 
                                'launchpad' , 
                                'flight_number' , 
                                'name' , 
                                'date_utc' , 
                                'date_unix' , 
                                'date_local' , 
                                'date_precision' , 
                                'upcoming' , 
                                'cores' , 
                                'auto_update' , 
                                'tbd' , 
                                'launch_library_id' , 
                                'id'
                                ]]


retained_feature_raw_data.head()




,net,window,rocket,success,failures,details,crew,ships,capsules,payloads,...,date_utc,date_unix,date_local,date_precision,upcoming,cores,auto_update,tbd,launch_library_id,id
0,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 33, 'altitude': None, 'reason': 'mer...",Engine failure at 33 seconds and loss of vehicle,[],[],[],[5eb0e4b5b6c3bb0006eeb1e1],...,2006-03-24,1143239400,2006-03-25T10:30:00+12:00,hour,False,"[{'core': '5e9e289df35918033d3b2623', 'flight'...",True,False,None,5eb87cd9ffd86e000604b32a
1,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 301, 'altitude': 289, 'reason': 'har...",Successful first stage burn and transition to ...,[],[],[],[5eb0e4b6b6c3bb0006eeb1e2],...,2007-03-21,1174439400,2007-03-21T13:10:00+12:00,hour,False,"[{'core': '5e9e289ef35918416a3b2624', 'flight'...",True,False,None,5eb87cdaffd86e000604b32b
2,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 140, 'altitude': 35, 'reason': 'resi...",Residual stage 1 thrust led to collision betwe...,[],[],[],"[5eb0e4b6b6c3bb0006eeb1e3, 5eb0e4b6b6c3bb0006e...",...,2008-08-03,1217734440,2008-08-03T15:34:00+12:00,hour,False,"[{'core': '5e9e289ef3591814873b2625', 'flight'...",True,False,None,5eb87cdbffd86e000604b32c
3,False,0.0,5e9d0d95eda69955f709d1eb,True,[],Ratsat was carried to orbit on the first succe...,[],[],[],[5eb0e4b7b6c3bb0006eeb1e5],...,2008-09-28,1222643700,2008-09-28T11:15:00+12:00,hour,False,"[{'core': '5e9e289ef3591855dc3b2626', 'flight'...",True,False,None,5eb87cdbffd86e000604b32d
4,False,0.0,5e9d0d95eda69955f709d1eb,True,[],None,[],[],[],[5eb0e4b7b6c3bb0006eeb1e6],...,2009-07-13,1247456100,2009-07-13T15:35:00+12:00,hour,False,"[{'core': '5e9e289ef359184f103b2627', 'flight'...",True,False,None,5eb87cdcffd86e000604b32e


we need to clean some data:

1. Choose the values where core = 1; selecting single core rockets
2. Choose the values where payloads = 1; selecting single Payload flights


In [59]:
to_be_fetched_data.to_csv("output.csv", index=False)